In [7]:
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing
%pylab inline

# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Populating the interactive namespace from numpy and matplotlib


In [8]:
# read police shooting data
shooting_df = pd.read_excel('datasets/MPVDatasetDownload.xlsx')
shooting_df.head()


,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Criminal Charges?,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Unnamed: 24
0,Name withheld by police,Unknown,Male,Unknown race,NaN,2018-09-18,300 35th Street,Virginia Beach,VA,23451.0,...,No known charges,https://wtkr.com/2018/09/18/virginia-beach-pol...,Unknown,Allegedly Armed,gun,attack,Foot,No,4033.0,NaN
1,Jose Alvarez,23,Male,Hispanic,https://www.fatalencounters.org/wp-content/upl...,2018-09-17,3466 Brinkley Rd,Temple Hills,MD,20748.0,...,No known charges,http://pgpolice.blogspot.com/2018/09/pgpd-inve...,No,Allegedly Armed,gun,other,Foot,No,4038.0,NaN
2,Susan Muller,54,Female,Unknown race,NaN,2018-09-17,59-14 69th Street,Flushing,NY,11378.0,...,No known charges,http://www.nydailynews.com/new-york/nyc-crime/...,Yes,Allegedly Armed,knife,attack,Not fleeing,Yes,4039.0,NaN
3,Name withheld by police,Unknown,Male,Unknown race,NaN,2018-09-17,13105 Greenville Rd,Hopkinsville,KY,42240.0,...,No known charges,https://www.wsmv.com/news/suspect-shot-killed-...,No,Vehicle,vehicle,attack,Car,No,4040.0,NaN
4,"Ricardo ""Macho"" Avenia",29,Male,Hispanic,NaN,2018-09-16,571 Pershing Ave,Lancaster,PA,17602.0,...,No known charges,https://lancasteronline.com/news/local/da-poli...,No,Allegedly Armed,gun,attack,Not fleeing,No,4035.0,NaN


#### CRIME DATASET Vars:
- index: Table Index
- EDITION: Edition Number
- PART: Part Number
- IDNO: SEQUENTIAL CASE ID 
- CPOPARST: COUNTY POPULATION-AGENCIES REPORT ARRESTS 
- CPOPCRIM: COUNTY POPULATION-AGENCIES REPORT CRIMES
- AGARRST: NUMBER OF AGENCIES IN COUNTY REPORT ARRESTS
- AGOFF: NUMBER OF AGENCIES IN COUNTY REPORT CRIMES
- COVIND: COVERAGE INDICATOR 
- INDEX: Total number of UCR (Uniform Crime Report) Index crimes, excluding arson. The sum of variables MURDER through MVTHEFT.
- MODINDX: MODIFIED INDEX. Total number of UCR (Uniform Crime Report) index crimes reported, including arson (sum of variables, MURDER through ARSON)
- MURDER: Number of MURDERS
- RAPE: Number of RAPES
- ROBBERY: Number of ROBBERIES
- AGASSLT: Number of AGGRAVATED ASSAULTS 
- BURGLRY: Number of BURGLARIES 
- LARCENY: Number of LARCENIES
- MVTHEFT: Number of MOTOR VEHICLE THEFTS 
- ARSON: Number of ARSONS 
- population: POPULATION 
- FIPSST: FIPS STATE CODE
- FIPSCTY: FIPS COUNTY CODE



In [9]:
# read county wise crime data
crime_df = pd.read_csv('datasets/2016 - united-states-crime-rates-by-county.csv')
crime_df.head()


,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31


In [10]:
zip_fip_df = pd.read_csv('datasets/ZIP-COUNTY-FIPS_2017-06.csv')
zip_fip_df.columns = ['Zipcode', 'COUNTYNAME', 'STATE', 'county_fips', 'CLASSFP']
zip_fip_df.head()

,Zipcode,COUNTYNAME,STATE,county_fips,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [11]:
# split county and state and merge it with crime
county_state = crime_df['county_name'].str.split(',', expand=True)
crime_df = crime_df.merge(county_state, how='outer', left_index=True, right_index=True)
crime_df = crime_df.merge(county_state, how='outer', left_index=True, right_index=True)

crime_df.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,0_x,1_x,0_y,1_y
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,13791,3543,464,318416,29,510,St. Louis city,MO,St. Louis city,MO
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,1753,189,28,49746,5,35,Crittenden County,AR,Crittenden County,AR
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,184,12,2,7629,17,3,Alexander County,IL,Alexander County,IL
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,4,4,0,412,48,261,Kenedy County,TX,Kenedy County,TX
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,494,60,0,27083,22,31,De Soto Parish,LA,De Soto Parish,LA


In [12]:
crime_df.drop(['0_y', '1_y'], axis=1, inplace=True)
crime_df.rename(index=str, columns={'0_x': 'county', '1_x': 'state'}, inplace=True)
crime_df.head()


,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,county,state
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,3609,4995,13791,3543,464,318416,29,510,St. Louis city,MO
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,662,1482,1753,189,28,49746,5,35,Crittenden County,AR
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,119,82,184,12,2,7629,17,3,Alexander County,IL
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,2,5,4,4,0,412,48,261,Kenedy County,TX
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,368,149,494,60,0,27083,22,31,De Soto Parish,LA


In [13]:
# crime_df[crime_df['county'].str.contains("Crittenden")]
crime_df[crime_df['FIPS_CTY'] == 3]
crime_df['county_fips'] = crime_df['FIPS_ST'] * 1000 + crime_df['FIPS_CTY']
crime_df.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,county,state,county_fips
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,4995,13791,3543,464,318416,29,510,St. Louis city,MO,29510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,1482,1753,189,28,49746,5,35,Crittenden County,AR,5035
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,82,184,12,2,7629,17,3,Alexander County,IL,17003
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,5,4,4,0,412,48,261,Kenedy County,TX,48261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,149,494,60,0,27083,22,31,De Soto Parish,LA,22031


In [14]:
# Shooting df has county fips code now. so can merge and group by on that.
shooting_df = shooting_df.merge(zip_fip_df, on="Zipcode", left_index=True, right_index=True)

In [40]:
# set(list(shooting_df['county_fips']))
# shooting_df[shooting_df['COUNTYNAME'].str.contains("Crittenden")]

# determine types of weapons

# set(list(shooting_df['Alleged Weapon (Source: WaPo)']))
# df = pd.DataFrame(shooting_df.groupby(['Alleged Weapon (Source: WaPo)'])['Alleged Weapon (Source: WaPo)'].count())
# df.sort_values(by=['Alleged Weapon (Source: WaPo)'])

# # determine causes of death
# ==> Not taking this
# df = pd.DataFrame(shooting_df.groupby(['Cause of death'])['Cause of death'].count())
# df.sort_values(by=['Cause of death'])

# # determine Official disposition of death (justified or other)
# df = pd.DataFrame(shooting_df.groupby(['Official disposition of death (justified or other)'])['Official disposition of death (justified or other)'].count())
# df.sort_values(by=['Official disposition of death (justified or other)'])

# # determine Official disposition of death (justified or other)
# df = pd.DataFrame(shooting_df.groupby(['Symptoms of mental illness?'])['Symptoms of mental illness?'].count())
# df.sort_values(by=['Symptoms of mental illness?'])

# # determine Official disposition of death (justified or other)
# leaving it because 6300 
# df = pd.DataFrame(shooting_df.groupby(['Criminal Charges?'])['Criminal Charges?'].count())
# df.sort_values(by=['Criminal Charges?'])

df = pd.DataFrame(shooting_df.groupby(['Body Camera (Source: WaPo)'])['Body Camera (Source: WaPo)'].count())
df.sort_values(by=['Body Camera (Source: WaPo)'])

# crime_df.columns
# crime_df.iloc[:, 13:]


,Body Camera (Source: WaPo)
Body Camera (Source: WaPo),
Bystander Video,1
Surveillance Video,2
Yes,355
No,3260


### Shooting and crime dataset has county_fips on each.

#### TODO:
- [x] classify crimes as violent crimes
    - add them
    - convert the required ones to crimes per 1000, or 10,000
- group by all in shooting dataset on county fips
- merge shooting and crime dataset
    - drop unnecessary columns

1. county level regressors ==> number of shootings
2. regressors ==> number of violent crimes
3. 


### other datasets
- race demographics
- poverty level
- unemployment level


- ? avg income?
- ? police spending
- ? number of police officers

### Shootings dataset
- count of shootings in that county, per 1000 residents
- percentage of total number of shootings, where victim was armed (of total number of shootings per county)
    - gun
    - knife
    - toy
    - undetermined
    - vehicle
    - others
- percentage of total number of shootings, where  victim fleeing (combine foot and car, so that eventually it's fleeing and not fleeing)
- percentage of total number of shootings, where Official disposition of death (justified or other) (of total number of shootings per county)
    - pending investigation
    - justified
    - unreported
    - others
- percentage of total number of shootings, where Symptoms of mental illness? (of total number of shootings per county)
    - yes
    - no
    - unknown
- XX police body camera (leave it for now. because implemented at some places and not at others)
   - yes
   - no

### Crime dataset
- amount of crime
- violent crimes, per 1000 residents
    - murder
    - rape
    - ag assault
    

- Merge all of them by county fips code

- Start playing with the data
    - correlations
    - modelling
        - see if quadratic terms are needed
        - See if PCA can be used
        - check for overfitting and look at IS R2 and OS R2.

- Idea:
    - DS1 and model1: aggregate county level shootings, merge it with demographics and do the analysis of demographics, poverty, unemployment on that
    - DS2 and model2: individual shootings, do the analysis of %armed, %mental illness etc


In [15]:
# Add violent crime stats to the dataset
try:
    crime_df.drop(['violent_crimes', 'violent_crimes_per_1000'])
except Exception:
    pass

crime_df['violent_crimes'] = crime_df['MURDER'] + crime_df['RAPE'] + crime_df['AGASSLT']
crime_df['violent_crimes_per_1000'] = crime_df['violent_crimes'] / crime_df['population'] * 1000

In [33]:
# zfill county_fips so that it's 5 characters
def to_zfilled_string_list(series, n):
    return [str(each).zfill(n) for each in list(series)]

crime_df['county_fips'] = to_zfilled_string_list(crime_df['county_fips'], 5)
shooting_df['county_fips'] = to_zfilled_string_list(shooting_df['county_fips'], 5)

In [37]:
# Dropping columns
drop_columns_shooting = [
    'URL of image of victim', 
    'Street Address of Incident', 
    'A brief description of the circumstances surrounding the death',
    'Link to news article or photo of official document',
    'WaPo ID (If included in WaPo database)',
    'Unnamed: 24',
    'COUNTYNAME',
    'STATE',
    'CLASSFP'
]
shooting_df.drop(drop_columns_shooting, axis=1, inplace=True)
shooting_df.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),City,State,Zipcode,County,Agency responsible for death,Cause of death,Official disposition of death (justified or other),Criminal Charges?,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),county_fips
0,Name withheld by police,Unknown,Male,Unknown race,2018-09-18,Virginia Beach,VA,23451.0,Virginia Beach,Virginia Beach Police Department,Gunshot,Pending investigation,No known charges,Unknown,Allegedly Armed,gun,attack,Foot,No,01001
1,Jose Alvarez,23,Male,Hispanic,2018-09-17,Temple Hills,MD,20748.0,Prince George's,Prince George's County Police Department,Gunshot,Pending investigation,No known charges,No,Allegedly Armed,gun,other,Foot,No,01001
2,Susan Muller,54,Female,Unknown race,2018-09-17,Flushing,NY,11378.0,Queens,New York Police Department,Gunshot,Pending investigation,No known charges,Yes,Allegedly Armed,knife,attack,Not fleeing,Yes,01001
3,Name withheld by police,Unknown,Male,Unknown race,2018-09-17,Hopkinsville,KY,42240.0,Christian,Christian County Sheriff’s Department,Gunshot,Pending investigation,No known charges,No,Vehicle,vehicle,attack,Car,No,01001
4,"Ricardo ""Macho"" Avenia",29,Male,Hispanic,2018-09-16,Lancaster,PA,17602.0,Lancaster,Lancaster Police Department,Gunshot,Pending investigation,No known charges,No,Allegedly Armed,gun,attack,Not fleeing,No,01001


In [53]:
# Convert No-Yes to 0-1

# Bystander Video, Surveillance Video ==> 1 (police will know they're being filmed)
# only 3 instances in this case
def dummy_var_prep(string_int_mapping, series):
    return [string_int_mapping.get(each, each) for each in list(series)]

string_int_mapping = {
    'Bystander Video': 1, 
    'No': 0, 
    'Surveillance Video': 1, 
    'Yes': 1
}
shooting_df['Body Camera (Source: WaPo)'] = dummy_var_prep(string_int_mapping, shooting_df['Body Camera (Source: WaPo)'])
# there are NaN in there ^^


string_int_mapping = {
    'Drug or alcohol use': 1, # can be mistaken for mental illness
    'No': 0, 
    'Yes': 1,
    'Unknown': 2,
    'Unknown ': 2,
    'Unkown': 2,
    'unknown': 2
}

shooting_df['Symptoms of mental illness?'] = dummy_var_prep(string_int_mapping, shooting_df['Symptoms of mental illness?'])
shooting_df['Symptoms of mental illness?'] = shooting_df['Symptoms of mental illness?'].fillna(2)


string_int_mapping = {
    'Not fleeing': 0, 
    'Car': 1,
    'Foot': 1,
    'Other': 2
}
shooting_df['Fleeing (Source: WaPo)'] = dummy_var_prep(string_int_mapping, shooting_df['Fleeing (Source: WaPo)'])
shooting_df['Fleeing (Source: WaPo)'] = shooting_df['Fleeing (Source: WaPo)'].fillna(2)

def charged_or_not(series):
    result = []
    for each in series:
        if "Charged" in each:
            result.append(1)
        elif 'No Criminal Charges' in each or \
            each.lower() in ['no', 'no known charges']:
            result.append(0)
        elif each == 'Pending investigation':
            result.append(2)
    return result

shooting_df['Criminal Charges?'] = charged_or_not(shooting_df['Criminal Charges?'])



{0, 1, 2}